## importação das bibliotecas

In [ ]:
pip install openmeteo-requests

In [ ]:
pip install requests-cache retry-requests numpy pandas

In [ ]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 764.0 kB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import requests

from datetime import datetime
#from translate import Translator
from deep_translator import GoogleTranslator

In [ ]:
import openmeteo_requests
import requests_cache

from retry_requests import retry

## Coleta de dados das APIS

### API date.nager

https://date.nager.at/Api

In [ ]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)

dates = []
days_of_week = []
names = []


for public_holiday in public_holidays:

    if public_holiday.get('global'):
        # Convertendo a data do feriado para um objeto datetime
        date_str = public_holiday['date']
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')

        # Obtendo o dia da semana
        day_of_week = date_obj.strftime('%A')

        # Armazenando os dados nas listas
        dates.append(date_str)
        days_of_week.append(day_of_week)
        names.append(public_holiday['name'])


df_public_holidays = pd.DataFrame({
    'Date': dates,
    'Day of Week': days_of_week,
    'Name': names
})

df_public_holidays


,Date,Day of Week,Name
0,2024-01-01,Monday,New Year's Day
1,2024-02-12,Monday,Carnival
2,2024-02-13,Tuesday,Carnival
3,2024-03-29,Friday,Good Friday
4,2024-03-31,Sunday,Easter Sunday
5,2024-04-21,Sunday,Tiradentes
6,2024-05-01,Wednesday,Labour Day
7,2024-05-30,Thursday,Corpus Christi
8,2024-09-07,Saturday,Independence Day
9,2024-10-12,Saturday,Our Lady of Aparecida


In [ ]:
df_public_holidays.to_csv('feriados_ano.csv', sep=',', index = False) #salvar pra abrir posteriormente

### Coleta de dados API 2 - Open meteo

https://open-meteo.com/

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"],
	"timezone": "America/Sao_Paulo"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)
#print(daily_dataframe)

Coordinates -22.88224983215332°N -43.07611083984375°E
Elevation 12.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s


In [ ]:
daily_dataframe

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean
0,2024-01-01 03:00:00+00:00,51.0,27.939999,21.840000,24.854582
1,2024-01-02 03:00:00+00:00,51.0,29.840000,22.990000,25.885834
2,2024-01-03 03:00:00+00:00,63.0,30.189999,23.189999,26.223335
3,2024-01-04 03:00:00+00:00,61.0,28.840000,23.539999,24.869164
4,2024-01-05 03:00:00+00:00,51.0,28.139999,23.189999,25.183752
...,...,...,...,...,...
209,2024-07-28 03:00:00+00:00,1.0,31.590000,16.740000,23.658747
210,2024-07-29 03:00:00+00:00,3.0,25.889999,17.389999,21.823334
211,2024-07-30 03:00:00+00:00,63.0,21.289999,17.889999,19.817083
212,2024-07-31 03:00:00+00:00,55.0,24.939999,17.590000,20.390001


In [ ]:
daily_dataframe.to_csv('clima_temperatura.csv', sep=',', index = False) #salvar pra abrir posteriormente

## Leitura dados baixados

In [ ]:
#df_public_holidays = pd.read_csv('/content/feriados_ano.csv', delimiter=',')
#df_public_holidays.head()

,Date,Day of Week,Name
0,2024-01-01,Monday,New Year's Day
1,2024-02-12,Monday,Carnival
2,2024-02-13,Tuesday,Carnival
3,2024-03-29,Friday,Good Friday
4,2024-03-31,Sunday,Easter Sunday


In [ ]:
#daily_dataframe = pd.read_csv('/content/clima_temperatura.csv', delimiter=',')
#daily_dataframe.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean
0,2024-01-01 03:00:00+00:00,51.0,27.939999,21.840000,24.854582
1,2024-01-02 03:00:00+00:00,51.0,29.840000,22.990000,25.885834
2,2024-01-03 03:00:00+00:00,63.0,30.189999,23.189999,26.223335
3,2024-01-04 03:00:00+00:00,61.0,28.840000,23.539999,24.869164
4,2024-01-05 03:00:00+00:00,51.0,28.140000,23.189999,25.183752


## Tradução

In [ ]:
# Função para traduzir o conteúdo de uma célula
def translate_cell(cell, source_lang='en', target_lang='pt'):
    return GoogleTranslator(source=source_lang, target=target_lang).translate(cell)

In [ ]:
# Aplicar a tradução
df_public_holidays['Day of Week'] = df_public_holidays['Day of Week'].apply(translate_cell)
df_public_holidays['Name'] = df_public_holidays['Name'].apply(translate_cell)

df_public_holidays

,Date,Day of Week,Name
0,2024-01-01,Segunda-feira,Dia de Ano Novo
1,2024-02-12,Segunda-feira,Carnaval
2,2024-02-13,Terça-feira,Carnaval
3,2024-03-29,Sexta-feira,Sexta-feira Santa
4,2024-03-31,Domingo,Domingo de Páscoa
5,2024-04-21,Domingo,Tiradentes
6,2024-05-01,Quarta-feira,Dia do Trabalho
7,2024-05-30,Quinta-feira,Corpo de Cristo
8,2024-09-07,Sábado,Dia da Independência
9,2024-10-12,Sábado,Nossa Senhora Aparecida


In [ ]:
#df_public_holidays.to_csv('feriados_ano_pt.csv', sep=',', index = False)

## Perguntas

### 1. Quantos feriados há no Brasil em todo o ano de 2024?

In [ ]:
total_feriados = df_public_holidays['Name'].shape[0]
print(f"1. Quantos feriados há no Brasil em todo o ano de 2024? {total_feriados}")

1. Quantos feriados há no Brasil em todo o ano de 2024? 13


### 2. Qual mês de 2024 tem o maior número de feriados?

talvez add o nome dos meses

In [ ]:
# 2. Quais meses de 2024 têm o maior número de feriados?
df_public_holidays['Month'] = pd.to_datetime(df_public_holidays['Date']).dt.month
feriados_por_mes = df_public_holidays['Month'].value_counts()
max_feriados = feriados_por_mes.max()
meses_com_mais_feriados = feriados_por_mes[feriados_por_mes == max_feriados].index.tolist()

print(f"2. Quais meses de 2024 têm o maior número de feriados? Meses: {meses_com_mais_feriados} com {max_feriados} feriados")

2. Quais meses de 2024 têm o maior número de feriados? Meses: [2, 3, 5, 11] com 2 feriados


### 3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?

In [ ]:
# 3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?
dias_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira']
feriados_dias_semana = df_public_holidays[df_public_holidays['Day of Week'].isin(dias_semana)]

print(f"3. Quantos feriados em 2024 caem em dias de semana? {len(feriados_dias_semana)}")

3. Quantos feriados em 2024 caem em dias de semana? 8


## Perguntas parte 2

In [ ]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 214 non-null    object 
 1   weather_code         214 non-null    float64
 2   temperature_2m_max   214 non-null    float64
 3   temperature_2m_min   214 non-null    float64
 4   temperature_2m_mean  214 non-null    float64
dtypes: float64(4), object(1)
memory usage: 8.5+ KB


### 4. Qual foi a temperatura média em cada mês?
Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.

In [ ]:

daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

# Extrair o ano e o mês da coluna 'date' para um novo DataFrame
daily_dataframe['year_month'] = daily_dataframe['date'].dt.to_period('M')

# Calcular a temperatura média para cada mês
monthly_avg_temp = daily_dataframe.groupby('year_month')['temperature_2m_mean'].mean().reset_index()

# Renomear as colunas para melhor entendimento
monthly_avg_temp.columns = ['Year-Month', 'Average Temperature']

print(f"Temperatura média em cada mês:")
print(monthly_avg_temp)


Temperatura média em cada mês:
  Year-Month  Average Temperature
0    2024-01            26.694234
1    2024-02            27.212988
2    2024-03            26.416479
3    2024-04            25.169930
4    2024-05            24.990739
5    2024-06            22.648958
6    2024-07            21.105591
7    2024-08            21.225414


<ipython-input-49-9db8e5f22c24>:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  daily_dataframe['year_month'] = daily_dataframe['date'].dt.to_period('M')


### 5. Qual foi o tempo predominante em cada mês nesse período?
Utilize como referência para o código de tempo (weather_code) o seguinte link: WMO Code.

In [ ]:
#Add o arquivo para ser aberto, encontra-se na pasta dados

with open('/content/descriptions.json', 'r') as file:
    json_data = json.load(file)

In [ ]:
# Transformar o dicionário em um DataFrame
df = pd.DataFrame({
    'cod': list(json_data.keys()),
    'day_description': [json_data[key]['day']['description'] for key in json_data],
    'night_description': [json_data[key]['night']['description'] for key in json_data]
})

print(df)

   cod                day_description              night_description
0    0                          Sunny                          Clear
1    1                   Mainly Sunny                   Mainly Clear
2    2                  Partly Cloudy                  Partly Cloudy
3    3                         Cloudy                         Cloudy
4   45                          Foggy                          Foggy
5   48                       Rime Fog                       Rime Fog
6   51                  Light Drizzle                  Light Drizzle
7   53                        Drizzle                        Drizzle
8   55                  Heavy Drizzle                  Heavy Drizzle
9   56         Light Freezing Drizzle         Light Freezing Drizzle
10  57               Freezing Drizzle               Freezing Drizzle
11  61                     Light Rain                     Light Rain
12  63                           Rain                           Rain
13  65                     Heavy R

In [ ]:
# Aplicar a tradução
df['day_description'] = df['day_description'].apply(translate_cell)
df['night_description'] = df['night_description'].apply(translate_cell)

df

,cod,day_description,night_description
0,0,Ensolarado,Claro
1,1,Principalmente ensolarado,Principalmente claro
2,2,Parcialmente nublado,Parcialmente nublado
3,3,Nublado,Nublado
4,45,Enevoado,Enevoado
5,48,Nevoeiro de geada,Nevoeiro de geada
6,51,Chuvisco Leve,Chuvisco Leve
7,53,Chuvisco,Chuvisco
8,55,Chuva forte,Chuva forte
9,56,Chuvisco leve congelante,Chuvisco leve congelante


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cod                28 non-null     object
 1   day_description    28 non-null     object
 2   night_description  28 non-null     object
dtypes: object(3)
memory usage: 800.0+ bytes


In [ ]:
# Converter a coluna 'cod' para o tipo float64
df['cod'] = df['cod'].astype(float)

In [ ]:
# Verificar os tipos das colunas
print(df.dtypes)

cod                  float64
day_description       object
night_description     object
dtype: object


In [ ]:
df.head()

,cod,day_description,night_description
0,0.0,Ensolarado,Claro
1,1.0,Principalmente ensolarado,Principalmente claro
2,2.0,Parcialmente nublado,Parcialmente nublado
3,3.0,Nublado,Nublado
4,45.0,Enevoado,Enevoado


necessário salvar o df

In [ ]:
#df.to_csv('classi_tempo_pt.csv', sep=',', index = False) #salvando a tradução

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cod                28 non-null     float64
 1   day_description    28 non-null     object 
 2   night_description  28 non-null     object 
dtypes: float64(1), object(2)
memory usage: 800.0+ bytes


In [ ]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 214 non-null    datetime64[ns, UTC]
 1   weather_code         214 non-null    float64            
 2   temperature_2m_max   214 non-null    float64            
 3   temperature_2m_min   214 non-null    float64            
 4   temperature_2m_mean  214 non-null    float64            
 5   year_month           214 non-null    period[M]          
dtypes: datetime64[ns, UTC](1), float64(4), period[M](1)
memory usage: 10.2 KB


In [ ]:
# Realizar o merge entre os DataFrames, combinando as colunas 'weather_code' e 'cod'
df_merged = pd.merge(daily_dataframe, df, left_on='weather_code', right_on='cod', how='left')

# Remover a coluna 'cod' do DataFrame final
df_merged = df_merged.drop(columns=['cod'])

df_merged.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,year_month,day_description,night_description
0,2024-01-01 03:00:00+00:00,51.0,27.939999,21.840000,24.854582,2024-01,Chuvisco Leve,Chuvisco Leve
1,2024-01-02 03:00:00+00:00,51.0,29.840000,22.990000,25.885834,2024-01,Chuvisco Leve,Chuvisco Leve
2,2024-01-03 03:00:00+00:00,63.0,30.189999,23.189999,26.223335,2024-01,Chuva,Chuva
3,2024-01-04 03:00:00+00:00,61.0,28.840000,23.539999,24.869164,2024-01,Chuva leve,Chuva leve
4,2024-01-05 03:00:00+00:00,51.0,28.140000,23.189999,25.183752,2024-01,Chuvisco Leve,Chuvisco Leve


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 214 non-null    datetime64[ns, UTC]
 1   weather_code         214 non-null    float64            
 2   temperature_2m_max   214 non-null    float64            
 3   temperature_2m_min   214 non-null    float64            
 4   temperature_2m_mean  214 non-null    float64            
 5   year_month           214 non-null    period[M]          
 6   day_description      214 non-null    object             
 7   night_description    214 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(4), object(2), period[M](1)
memory usage: 13.5+ KB


In [ ]:
# Encontrar o tempo predominante em cada mês
predominant_weather = df_merged.groupby('year_month')['day_description'].agg(lambda x: x.value_counts().idxmax())


print(predominant_weather)

year_month
2024-01                        Chuva
2024-02                Chuvisco Leve
2024-03                Chuvisco Leve
2024-04                      Nublado
2024-05                Chuvisco Leve
2024-06    Principalmente ensolarado
2024-07                      Nublado
2024-08         Parcialmente nublado
Freq: M, Name: day_description, dtype: object


### 6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?



In [ ]:
df_public_holidays

,Date,Day of Week,Name,Month
0,2024-01-01,Segunda-feira,Dia de Ano Novo,1
1,2024-02-12,Segunda-feira,Carnaval,2
2,2024-02-13,Terça-feira,Carnaval,2
3,2024-03-29,Sexta-feira,Sexta-feira Santa,3
4,2024-03-31,Domingo,Domingo de Páscoa,3
5,2024-04-21,Domingo,Tiradentes,4
6,2024-05-01,Quarta-feira,Dia do Trabalho,5
7,2024-05-30,Quinta-feira,Corpo de Cristo,5
8,2024-09-07,Sábado,Dia da Independência,9
9,2024-10-12,Sábado,Nossa Senhora Aparecida,10


In [ ]:
df_public_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         13 non-null     object
 1   Day of Week  13 non-null     object
 2   Name         13 non-null     object
 3   Month        13 non-null     int32 
dtypes: int32(1), object(3)
memory usage: 492.0+ bytes


In [ ]:
df_merged['date'] = pd.to_datetime(df_merged['date'])

df_merged['date'] = df_merged['date'].dt.date

df_merged.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,year_month,day_description,night_description
0,2024-01-01,51.0,27.939999,21.840000,24.854582,2024-01,Chuvisco Leve,Chuvisco Leve
1,2024-01-02,51.0,29.840000,22.990000,25.885834,2024-01,Chuvisco Leve,Chuvisco Leve
2,2024-01-03,63.0,30.189999,23.189999,26.223335,2024-01,Chuva,Chuva
3,2024-01-04,61.0,28.840000,23.539999,24.869164,2024-01,Chuva leve,Chuva leve
4,2024-01-05,51.0,28.140000,23.189999,25.183752,2024-01,Chuvisco Leve,Chuvisco Leve


In [ ]:
df_public_holidays['Date'] = pd.to_datetime(df_public_holidays['Date'])
df_merged['date'] = pd.to_datetime(df_merged['date'])

In [ ]:
start_date = '2024-01-01'
end_date = '2024-08-01'

# Filtrar o DataFrame de feriados para incluir apenas as datas dentro do intervalo
df_public_holidays_filtered = df_public_holidays[(df_public_holidays['Date'] >= start_date) & (df_public_holidays['Date'] <= end_date)]

# Mesclar os DataFrames com base na data
df_merged_feriado = pd.merge(df_public_holidays_filtered, df_merged, left_on='Date', right_on='date', how='left')

# Selecionar as colunas de interesse
df_merged_feriado = df_merged_feriado[['Date', 'Name', 'temperature_2m_mean', 'day_description']]


df_merged_feriado

,Date,Name,temperature_2m_mean,day_description
0,2024-01-01,Dia de Ano Novo,24.854582,Chuvisco Leve
1,2024-02-12,Carnaval,30.156668,Principalmente ensolarado
2,2024-02-13,Carnaval,30.531670,Chuvisco Leve
3,2024-03-29,Sexta-feira Santa,25.044168,Chuva leve
4,2024-03-31,Domingo de Páscoa,24.531664,Chuvisco
5,2024-04-21,Tiradentes,23.087920,Parcialmente nublado
6,2024-05-01,Dia do Trabalho,28.108751,Ensolarado
7,2024-05-30,Corpo de Cristo,20.779580,Chuvisco Leve


### 7. Considere as seguintes suposições:



*   O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
*  Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
*  O cidadão carioca só vai à praia quando não está com frio;
*  O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).

Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?




In [ ]:
# Critérios de avaliação
def is_not_enjoyable_day(temp, description):
    # Verifica se é um dia frio (temperatura média menor que 20°C)
    if temp < 20:
        return True
    # Verifica se o clima é desfavorável para ir à praia (chuva ou nublado)
    if description not in ["Ensolarado", "Principalmente ensolarado"]:
        return True
    return False

# Verificar se os feriados são "não aproveitáveis"
df_merged_feriado["not_enjoyable"] = df_merged_feriado.apply(
    lambda row: is_not_enjoyable_day(row["temperature_2m_mean"], row["day_description"]),
    axis=1
)

# Filtrar feriados "não aproveitáveis"
non_enjoyable_holidays = df_merged_feriado[df_merged_feriado["not_enjoyable"]]

# Exibir os feriados "não aproveitáveis"
print("Feriados não aproveitáveis em 2024:")
print(non_enjoyable_holidays[["Date", "Name", "temperature_2m_mean", "day_description"]])


Feriados não aproveitáveis em 2024:
        Date               Name  temperature_2m_mean       day_description
0 2024-01-01    Dia de Ano Novo            24.854582         Chuvisco Leve
2 2024-02-13           Carnaval            30.531670         Chuvisco Leve
3 2024-03-29  Sexta-feira Santa            25.044168            Chuva leve
4 2024-03-31  Domingo de Páscoa            24.531664              Chuvisco
5 2024-04-21         Tiradentes            23.087920  Parcialmente nublado
7 2024-05-30    Corpo de Cristo            20.779580         Chuvisco Leve


### 8. Qual foi o feriado "mais aproveitável" de 2024?

Considere o melhor par tempo e temperatura.

In [ ]:
def get_enjoyability_score(temp, description):
    # Score baseado nas condições de aproveitabilidade
    score = 0
    if description == "Ensolarado":
        score += 2  # Melhor condição
    elif description == "Principalmente ensolarado":
        score += 1  # Boa condição

    if temp >= 20:
        score += 2  # Adiciona 2 pontos se a temperatura é agradável (não está frio)
    elif temp >= 25:
        score += 1  # Adiciona 1 ponto se a temperatura está muito agradável

    return score

# Adicionar uma coluna de pontuação
df_merged_feriado["enjoyability_score"] = df_merged_feriado.apply(
    lambda row: get_enjoyability_score(row["temperature_2m_mean"], row["day_description"]),
    axis=1
)

# Identificar o feriado "mais aproveitável" com a maior pontuação
most_enjoyable_holiday = df_merged_feriado.loc[df_merged_feriado["enjoyability_score"].idxmax()]


print("\nFeriado mais aproveitável em 2024:")
print(most_enjoyable_holiday[["Date", "Name", "temperature_2m_mean", "day_description"]])



Feriado mais aproveitável em 2024:
Date                   2024-05-01 00:00:00
Name                       Dia do Trabalho
temperature_2m_mean              28.108751
day_description                 Ensolarado
Name: 6, dtype: object
